In [1]:
from collections import defaultdict
from mysql.connector.errors import Error
from itertools import islice

import datetime
import sys
import json
import mysql.connector
import os
import glob
import re


In [ ]:
d=defaultdict(list)
fileslist=[]
errorlist=[
          '/media/data/sensemaking/dataset/Vansterpartiet/*.dat',
          '/media/data/sensemaking/dataset/Sveriagedemokraterna/*.dat',
          '/media/data/sensemaking/dataset/Socialdemokraterna/*.dat',
          '/media/data/sensemaking/dataset/Piratpartiet/*.dat',
          ]

dirlist= ['/media/data/sensemaking/dataset/centralpartiet/clean/*.dat',
          '/media/data/sensemaking/dataset/FeministisktInitiativ/clean/*.dat',
          '/media/data/sensemaking/dataset/Kristdemokraterna/clean/*.dat',
          '/media/data/sensemaking/dataset/Liberalerna/clean/*.dat',
          '/media/data/sensemaking/dataset/Vansterpartiet/*.dat',
          '/media/data/sensemaking/dataset/Sveriagedemokraterna/*.dat',
          '/media/data/sensemaking/dataset/Socialdemokraterna/*.dat',
          '/media/data/sensemaking/dataset/Piratpartiet/*.dat'
         ]

for dirl in dirlist:
    fileslist.extend(glob.glob(dirl,recursive=True))
    print(re.search('dataset/(.+?)/',dirl).group(1))
    print('directory files are adding to filelist: %s:'%dirl)
print('no.of file in directory: %d'%len(fileslist))

    

In [ ]:
tweet_list=[]
tweet_list1=[]
print("3rd block strated")
# each file listed in fileslist 
for file in fileslist:
    #print(file)
    with open(file) as f:
        content = f.readlines()
         # you may also want to remove whitespace characters like `\n` at the end of each line
    #content = [x.strip() for x in content]
    #print(len(content))
       
    for x in content:
           
        try:
            if x.strip().endswith('}'):
                jsondata = json.loads(x,encoding='latin1')
            else:
                
                continue
                #   jsondata = json.loads(x,encoding='latin1')
                    
        except ValueError as ex:
            #print(f)
           
            continue
            #raise ValueError(ex)
        #tweet-table data    
        tweet_id= jsondata['id']
        tweet_list.append(tweet_id)
tweet_list1= list(set(tweet_list))
print(len(tweet_list))
print(len(tweet_list1))

for i in range(len(tweet_list1)):
    if i==73 or i== 199842:
        print(tweet_list1[i])
    
    

s1= helpinsert(tweet_list1)
#s1= tinsert(tweet_list1)
print('help table insertion operation completed: %s'%s1)
    
  

In [ ]:
def tinsert(tweet_list1):
    
    print("tinsert method started")
    print(len(tweet_list1))
       
        
    cnx = mysql.connector.connect(user='root', password= 'root', host = 'localhost',database = 'twitterSchema')
    
    q= "insert into t(tw,li)  select ? , GROUP_CONCAT(CONCAT(hashtag.text)) as ht from tweet inner join tweets_hashtag using(tweet_id) inner join hashtag on tweets_hashtag.hash_id= hashtag.hashtag_id where tweet_id=?" 
    print(q)
    try:
        cursor = cnx.cursor(prepared=True)                        
        for i in range(len(tweet_list1)):
            print(i)
            try:
                    cursor.execute(q,(tweet_list1[i],tweet_list1[i]))
            except Error as err:
                print("Something went wrong: {}")
                print(i)
            
        cnx.commit()
        cursor.close()
        cnx.close()
    except Error as err:
        print("Something went wrong: {}".format(err))
      
        raise mysql.connector.Error(err)
        
    return 'success';

In [ ]:
def helpinsert(tweet_list1):
    
    print("helpinsert method started")
    print(len(tweet_list1))
       
        
    cnx = mysql.connector.connect(user='root', password= 'root', host = 'localhost',database = 'twitterSchema')
    
    q= "insert into help(tweet_id,text,uid,created,uname)  select tweet.tweet_id, tweet.text,users.id_str,tweet.created_at,users.name from users inner join tweet using(id_str) where tweet_id=?" 
    print(q)
    try:
        cursor = cnx.cursor(prepared=True)                        
        for i in range(len(tweet_list1)):
            print(i)
            try:
                    cursor.execute(q,(tweet_list1[i],))
            except Error as err:
                print("Something went wrong: {}")
                print(i)
            
        cnx.commit()
        cursor.close()
        cnx.close()
    except Error as err:
        print("Something went wrong: {}".format(err))
      
        raise mysql.connector.Error(err)
        
    return 'success';

In [2]:
""" users hashatg realted files uhashtag table contains userid,hahstag,created_at,parties"""
htlist=[]
cnx = mysql.connector.connect(user='root', password= 'root', host = 'localhost',database = 'twitterSchema')
q= "select * from t" 
p="insert into userhashtag(userid,ht,created_at,pname,tweet_id) select id_str,?,date(created_at),party_name,? from tweet inner join mentions_party using(tweet_id) where tweet_id=?"

try:
    cursor = cnx.cursor(prepared=True)    
    cursor.execute(q)
    rows = cursor.fetchall() #return list of tuples (tweetid,hashatg(svpol,ligpol,.))
   
    print(len(rows))
   
    for x,y in rows:
        y=y.decode("utf-8")
        ht=y.split(",")      
        for i in ht:
            htlist.append((i,x,x))
    print(len(htlist))
    for i in range(20):
        print(htlist[i])
            
    for i in range(len(htlist)):
         cursor.execute(p,htlist[i])
       
       
    cnx.commit()
    cursor.close()
    cnx.close()
except Error as err:
    print("Something went wrong: {}".format(err))
    raise mysql.connector.Error(err)
print("it wass succes::::")

594613
927894
('svpol', 723152822133493760, 723152822133493760)
('nato', 741373653951090688, 741373653951090688)
('föpol', 741373653951090688, 741373653951090688)
('säkpol', 741373653951090688, 741373653951090688)
('svpol', 687165368431947776, 687165368431947776)
('flygskatt', 803997605659344896, 803997605659344896)
('ssu16', 711219964733202433, 711219964733202433)
('svpol', 803699622363414533, 803699622363414533)
('svpol', 753530885794893825, 753530885794893825)
('svpol', 734468755116462080, 734468755116462080)
('svtagenda', 734468755116462080, 734468755116462080)
('svpol', 764487277980618752, 764487277980618752)
('svpol', 697840533385912320, 697840533385912320)
('noterad', 739130323586273280, 739130323586273280)
('dn', 739130323586273280, 739130323586273280)
('svpol', 810739295392858112, 810739295392858112)
('svpol', 745529639846612996, 745529639846612996)
('flygskatt', 803988293998313472, 803988293998313472)
('svpol', 803988293998313472, 803988293998313472)
('eupol', 789219568589864

In [8]:
""" users hashatg realted files userhashtag table contains userid,hahstag,created_at,parties"""
htlist=[]
dummy=[]
cnx = mysql.connector.connect(user='root', password= 'root', host = 'localhost',database = 'twitterSchema')
q= "select  hashtag_id,text from hashtag" 
p="select m,count(distinct userid),count(distinct tweet_id)as ctweet,@total:=@total+count(distinct tweet_id) as cumtweet from (select userid,tweet_id,month(created_at) as m from userhashtag cross join (select @total := 0) as var where ht=?)a  group by m"


try:
    cursor = cnx.cursor(prepared=True)    
    cursor.execute(q)
    rows = cursor.fetchall() #return list of tuples (tweetid,hashatg(svpol,ligpol,.))
    
    for x,y in rows:
        y1=y.decode("utf-8")
        htlist.append((y1)) 
except Error as err:
    print("Something went wrong: {}".format(err))
    raise mysql.connector.Error(err)
print("it wass succes::::")

it wass succes::::


In [9]:
print(len(htlist))

10090


In [10]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [16]:
htlist_chunks=list(chunks(range(0,10090), 1000))
print(htlist_chunks)

[range(0, 1000), range(1000, 2000), range(2000, 3000), range(3000, 4000), range(4000, 5000), range(5000, 6000), range(6000, 7000), range(7000, 8000), range(8000, 9000), range(9000, 10000), range(10000, 10090)]


In [17]:
for i in htlist_chunks[1]:
        cursor.execute(p,(htlist[i],))
        rows1=cursor.fetchall()
        for j in rows1:
            dummy.append([htlist[i],j[0],j[1],j[2],j[3]])  #[()]   
for j in range(0,20):
        print(dummy[j])
        
print(len(dummy))

cursor.close()
cnx.commit()
cnx.close()
print("it wass succes::::")                      

AttributeError: 'NoneType' object has no attribute 'cmd_stmt_reset'